## Predicting Churn Rate using ANN


We are given bank customers data which suggests if the customer with certain properties left the bank or not. 

The challenge here is to create a model so that we can predict if a customer with given properties will leave the bank or not.

We do this by creating an ANN and training on it.

### Step 1: Prepare the data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('data/Churn_Modelling.csv')

In [3]:
dataset.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [4]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


* 'Exited' is the lable that we want to predict
* We have Surname, Geography, Gender, Exited as categorical variables - they need to be encoded.
* Surname will have no bearing on our prediction. We need to delete it.
* All values need to be normalized for usage in NN

In [5]:
# Drop Surname
dataset = dataset.drop(['Surname', 'RowNumber', 'CustomerId'], axis = 1)

In [6]:
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
# Encode the categorical variables
dataset = pd.get_dummies(dataset)
dataset.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [8]:
# To avoid the dummy variable trap, we need to delete 1 column for each of the dummy variable column collection
dataset = dataset.drop('Geography_Spain', axis = 1)
dataset = dataset.drop('Gender_Male', axis = 1)
dataset.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Gender_Female
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,1
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,1
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,1
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [9]:
# Split the data into dependent and independent variables
y = dataset['Exited'].values
dataset = dataset.drop(['Exited'], axis = 1)
X = dataset.iloc[:, :].values

In [10]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Step 2: Create the NN and train the model 

(Without using KFold cross validation. That is done in Step 2a.)

We will create a model with two hidden layers (with 6 nodes each)

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using Theano backend.
Using cuDNN version 5110 on context None
Mapped name None to device cuda: GeForce GTX 1080 Ti (0000:23:00.0)


In [13]:
# Create the NN (it is classifier for us)
classifier = Sequential()

In [14]:
# Create the layers
classifier.add(Dense(units = 6, activation = 'relu', kernel_initializer='uniform', input_dim=11))
classifier.add(Dense(units = 6, activation = 'relu', kernel_initializer='uniform'))
classifier.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer='uniform'))

In [15]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [16]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 0s - loss: 0.4874 - acc: 0.7921     
Epoch 2/100
8000/8000 [==============================] - 0s - loss: 0.4241 - acc: 0.7927     
Epoch 3/100
8000/8000 [==============================] - 0s - loss: 0.4140 - acc: 0.8017     
Epoch 4/100
8000/8000 [==============================] - 0s - loss: 0.4042 - acc: 0.8266     
Epoch 5/100
8000/8000 [==============================] - 0s - loss: 0.3904 - acc: 0.8304     
Epoch 6/100
8000/8000 [==============================] - 0s - loss: 0.3816 - acc: 0.8385     
Epoch 7/100
8000/8000 [==============================] - 0s - loss: 0.3770 - acc: 0.8430     
Epoch 8/100
8000/8000 [==============================] - 0s - loss: 0.3734 - acc: 0.8470     
Epoch 9/100
8000/8000 [==============================] - 0s - loss: 0.3721 - acc: 0.8459     
Epoch 10/100
8000/8000 [==============================] - 0s - loss: 0.3694 - acc: 0.8484     
Epoch 11/100
8000/8000 [==============================] - 0

8000/8000 [==============================] - 0s - loss: 0.3436 - acc: 0.8567     
Epoch 87/100
8000/8000 [==============================] - 0s - loss: 0.3434 - acc: 0.8566     
Epoch 88/100
8000/8000 [==============================] - 0s - loss: 0.3436 - acc: 0.8581     
Epoch 89/100
8000/8000 [==============================] - 0s - loss: 0.3429 - acc: 0.8590     
Epoch 90/100
8000/8000 [==============================] - 0s - loss: 0.3433 - acc: 0.8569     
Epoch 91/100
8000/8000 [==============================] - 0s - loss: 0.3430 - acc: 0.8571     
Epoch 92/100
8000/8000 [==============================] - 0s - loss: 0.3431 - acc: 0.8579     
Epoch 93/100
8000/8000 [==============================] - 0s - loss: 0.3434 - acc: 0.8592     
Epoch 94/100
8000/8000 [==============================] - 0s - loss: 0.3432 - acc: 0.8569     
Epoch 95/100
8000/8000 [==============================] - 0s - loss: 0.3433 - acc: 0.8582     
Epoch 96/100
8000/8000 [==============================] - 0s - 

### Step 3: Evaluate the model

In [17]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print (cm)

[[1526   95]
 [ 165  214]]


In [18]:
# Use Accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.87

### Step 4: How would the accuracy be with a traditional classifier?

In [19]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.83099999999999996

In [20]:
# SVC
from sklearn.svm import SVC
sv = SVC()
sv.fit(X_train, y_train)
sv.score(X_test, y_test)

0.87250000000000005

In [21]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

0.85250000000000004

### Step 5: Make a prediction

Use our ANN model to predict if the customer with the following informations will leave the bank: 

* Geography: France
* Credit Score: 600
* Gender: Male
* Age: 40 years old
* Tenure: 3 years
* Balance: \$60000
* Number of Products: 2
* Does this customer have a credit card ? Yes
* Is this customer an Active Member: Yes
* Estimated Salary: \$50000
* So should we say goodbye to that customer ?

In [22]:
X_test.shape

(2000, 11)

In [23]:
type(X_test)

numpy.ndarray

In [24]:
to_pred = sc.transform([[600, 40, 3, 60000, 2, 1, 1, 50000, 1, 0, 0]])
ip = np.array(to_pred)
ip

array([[-0.52659927,  0.0976982 , -0.69360377, -0.26828478,  0.81602636,
         0.64512003,  0.97043679, -0.87938236,  0.99476371, -0.57869742,
        -0.91670588]])

In [25]:
pred = classifier.predict(ip)

In [26]:
pred > 0.5

array([[False]], dtype=bool)

Answer: Result is False (Value of Exit?). So customer doesn't leave the bank.

### Addendum 1: Create the NN and train the model  using KFold Cross Validation

Let's now try to apply cross validation to the training of the model to make sure our bias and variance are within acceptable limits.

In [27]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, activation = 'relu', kernel_initializer='uniform', input_dim=11))
    classifier.add(Dense(units = 6, activation = 'relu', kernel_initializer='uniform'))
    classifier.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer='uniform'))
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier_kfold = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=100)
accuracies = cross_val_score(estimator=classifier_kfold, X=X_train, y=y_train, cv=10, n_jobs=-1)

JoblibGpuArrayException: JoblibGpuArrayException
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/anand/miniconda3/envs/py35/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.5/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home

In [ ]:
accuracies

In [ ]:
np.var(accuracies)

In [ ]:
np.mean(accuracies)

### Addendum 2: Apply dropout regularization (solution to overfitting)

* Dropout regularization is a solution to taking care of overfitting
* At each iteration of the training, some neurons of the ANN are randomly disabled to prevent them from being too dependent on each other when they learn the correlations. By overriding the neurons, the ANN learns several independent correlations in the data, because each time we get a new configuration of the data.

In [ ]:
from keras.layers import Dropout

In [ ]:
# Create the NN (it is classifier for us)
classifier_dropout = Sequential()

In [ ]:
# Create the layers
classifier_dropout.add(Dense(units = 6, activation = 'relu', kernel_initializer='uniform', input_dim=11))
classifier_dropout.add(Dropout(rate = 0.1))
classifier_dropout.add(Dense(units = 6, activation = 'relu', kernel_initializer='uniform'))
classifier_dropout.add(Dropout(rate = 0.1))
classifier_dropout.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer='uniform'))

In [ ]:
classifier_dropout.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier_dropout.fit(X_train, y_train, batch_size=10, epochs=100)

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
# Use Accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

### Addendum 3: Parameter tuning

One can use GridSearchCV to get the best combination of hyper parameters (Implemented below)

### Putting it all together

* Cross Validation
* Dropout Regularization
* Parameter Tuning

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def build_classifier(opti):
    classifier = Sequential()
    classifier.add(Dense(units = 6, activation = 'relu', kernel_initializer='uniform', input_dim=11))
    classifier_dropout.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, activation = 'relu', kernel_initializer='uniform'))
    classifier_dropout.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer='uniform'))
    classifier.compile(optimizer=opti, loss='binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier_final = KerasClassifier(build_fn=build_classifier, verbose=0)

parameters = {'batch_size': [16, 32],
              'epochs' : [100, 500],
              'opti': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator=classifier_final, 
                           param_grid=parameters,
                           scoring='accuracy',
                           cv=10,
                           n_jobs=-1)

grid_search = grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

print(best_params)
print(best_accuracy)
